### 一、完整流程总结

#### 1、准备数据
#### 2、数据可视化
#### 3、数据和模型管道搭建
先求数据集有多长

然后生成索引列表

索引列表要打散

再来创建batch索引生成器

index_select要用LongTensor

这里就该转换了

indexes记得要用min

避免最后一个batch不够数

返回要用yield

index_select还要选维度

要选最大维度才可以
#### 4、定义模型
参数定义

正向传播

损失函数

准确率预测
#### 5、定义单步训练
正向传播求估值

估值标签求损失

反向传播求梯度

不记梯度改参数

梯度归零下一轮
#### 6、训练模型
先定要for多少epoch

再for每个mini_batch

循环里面主要放单步

记得返回计算损失值

每当完成一个epoch

打印信息看看怎么样
#### 7、结果可视化

In [37]:
import torch
import numpy as np
from torchkeras import KerasModel 
from torchkeras import summary

### 二、tensor数据生成

#### 1、固定正态分布随机数
#### torch.randn()

均值为0，标准差为1的随机数，参数依次为
shape,数据类型，是否需要导数

In [22]:
X = torch.randn((2,3,5),dtype=torch.float32,requires_grad= True)
print(X)

tensor([[[ 1.8693, -0.1101, -0.0944, -0.5898, -1.2877],
         [-0.2041,  0.4701,  2.1172,  0.1126, -1.3039],
         [ 0.1538,  1.4080, -0.3981,  0.2262,  0.4479]],

        [[-0.3301, -0.0203, -1.0164,  0.6360, -0.0439],
         [ 0.0301,  0.1619,  2.0155,  1.0248, -0.6261],
         [ 0.5696,  0.0786,  1.3952, -1.3562,  0.1008]]], requires_grad=True)


#### 2、指定正态分布随机数
#### torch.normal()，传入参数包含两种形式。

①分别给定平均值和方差的tensor。mean和std的元素数量必须相同，如果满足数量相同，但是形状不相同，最终输出的元素形状以mean，也就是传入的第一个tensor为准。

②分别给定两个float以及一个元祖。分别表示mean和std的数值，以及输出的形状

In [23]:
#①给定平均值和方差的tensor，两者元素数量相同，但是形状不同，能运行，但是会提醒

torch.normal(torch.zeros(3,5),torch.ones(5,3))

C:\Users\Administrator\AppData\Local\Temp/ipykernel_10872/154419691.py:3: UserWarning: std and mean have the same number of elements, but are not broadcastable. This was previously a supported mode of operation, but is now deprecated and the support will be removed in version 1.6 release. Note that the current implementation reshapes std to the shape of mean, which may be incur data copies. Please ensure that std and mean are broadcastable to avoid these issues. (Triggered internally at  ..\aten\src\ATen/native/DistributionTemplates.h:191.)
  torch.normal(torch.zeros(3,5),torch.ones(5,3))


tensor([[-0.1581, -0.3717,  0.2744,  0.2189,  1.2002],
        [ 0.4153, -2.2840,  0.6194, -0.7958, -0.5260],
        [-0.3098,  1.7093,  0.6096,  0.0840, -0.1748]])

In [24]:
#②满足平均值1，方差3，形状（3,5）的随机数组

torch.normal(1,3,(3,5))

tensor([[ 4.7387,  3.8171,  4.5361,  0.7913,  3.3621],
        [ 4.3505,  4.0955, -2.5083, -0.6528,  2.6160],
        [ 2.4690, -5.2908,  3.6296,  2.0837,  3.5731]])

#### 3、numpy对象和tensor对象相互转换  
#### tensor.numpy()， torch.from_numpy(ndarray)

①tensor转numpy，共享内存：

tensor.numpy()。无参数,tensor对象有grads时使用

tensor.detach().numpy()。无参数，tensor对象没有grads时使用

②numpy转tensor，共享内存：

torch.from_numpy(ndarray)

In [52]:
'''
①此处X包含grads，如果直接用.numpy()进行转换，
torch库会提示，需要对tensor先使用detach()，之后才能转换成numpy对象
'''
#X.detach()会返回一个新的tensor，所以更新X也不会导致Z变化
Z = X.detach().numpy()
print(type(X))
print(type(Z))

<class 'torch.Tensor'>
<class 'numpy.ndarray'>


In [77]:
#tensor转numpy,不共享内存
A= torch.tensor([1.,2.,3.])
print('A:',A)
Z= A.numpy()
print('Z:',Z,'\n')

A[0]=5
print('A:',A)
print('Z:',Z)

A: tensor([1., 2., 3.])
Z: [1. 2. 3.] 

A: tensor([5., 2., 3.])
Z: [5. 2. 3.]


In [78]:
'''
②numpy转tensor，共享内存,改变原来的numpy数据时，tensor会一起更新
'''
#将numpy数据Z转换为tensor数据
Y = torch.from_numpy(Z)
print('Y:',Y)

#修改Z的值
Z[0]=10 
#Y的值将一起被调整
print('Y:',Y,'\n')

Y: tensor([5., 2., 3.])
Y: tensor([10.,  2.,  3.]) 



### 三、tensor数据操作

#### 1、增加和删除 
#### tensor数据的合并和切分  torch.cat(), torch.stack(), torch.split()

①torch.cat()。对形状相同的tensor进行合并，且不增加维度。第一个参数是待合并tensor的集合，第二个参数是合并的维度，默认是0维，维度不能超过原有维度的范围。

②torch.stack()。像栈的模式一样，对于形状相同的tensor进行合并，且增加维度。第一个参数是tensor的集合，第二个参数是合并的维度，默认是0维，维度最多可以在原有范围基础上增加1。

③torch.split()。将tensor进行分隔，不改变形状。第一个参数是tensor，第二个参数是整数或者整数列表，代表需要划分的个数，第三个参数是进行划分操作的维度，默认是0

In [6]:
'''
①torch.cat()。对形状相同的tensor进行合并，且不增加维度。
第一个参数是待合并tensor的集合，
第二个参数是合并的维度，默认是0维。
'''
print('X shape:',X.shape,'\n')
#对2个X在0维进行cat合并
print(torch.cat((X,X),0))
print(torch.cat((X,X),0).shape,'\n')

X shape: torch.Size([3, 5]) 

tensor([[ 0.2569,  1.3447, -1.0964, -0.7743,  0.4888],
        [-0.3436, -0.7069,  0.2946, -0.0270, -0.3178],
        [ 0.2489, -0.2260, -0.0092, -0.4739,  0.2413],
        [ 0.2569,  1.3447, -1.0964, -0.7743,  0.4888],
        [-0.3436, -0.7069,  0.2946, -0.0270, -0.3178],
        [ 0.2489, -0.2260, -0.0092, -0.4739,  0.2413]], grad_fn=<CatBackward0>)
torch.Size([6, 5]) 



In [13]:
#对2个X在1维进行cat合并
print(torch.cat((X,X),1))
print(torch.cat((X,X),1).shape,'\n')

tensor([[ 0.2569,  1.3447, -1.0964, -0.7743,  0.4888,  0.2569,  1.3447, -1.0964,
         -0.7743,  0.4888],
        [-0.3436, -0.7069,  0.2946, -0.0270, -0.3178, -0.3436, -0.7069,  0.2946,
         -0.0270, -0.3178],
        [ 0.2489, -0.2260, -0.0092, -0.4739,  0.2413,  0.2489, -0.2260, -0.0092,
         -0.4739,  0.2413]], grad_fn=<CatBackward0>)
torch.Size([3, 10]) 



In [14]:
'''
②torch.stack()。
像栈的模式一样，对于形状相同的tensor进行合并，且增加维度。
第一个参数是tensor的集合，第二个参数是合并的维度，默认是0维
'''

print('X shape:',X.shape,'\n')
#对2个X在0维进行stack合并
print(torch.stack((X,X),0))
print(torch.stack((X,X),0).shape,'\n')

X shape: torch.Size([3, 5]) 

tensor([[[ 0.2569,  1.3447, -1.0964, -0.7743,  0.4888],
         [-0.3436, -0.7069,  0.2946, -0.0270, -0.3178],
         [ 0.2489, -0.2260, -0.0092, -0.4739,  0.2413]],

        [[ 0.2569,  1.3447, -1.0964, -0.7743,  0.4888],
         [-0.3436, -0.7069,  0.2946, -0.0270, -0.3178],
         [ 0.2489, -0.2260, -0.0092, -0.4739,  0.2413]]],
       grad_fn=<StackBackward0>)
torch.Size([2, 3, 5]) 



In [21]:
#对2个X在2维进行stack合并
print(torch.stack((X,X),1))
print(torch.stack((X,X),1).shape,'\n')

tensor([[[ 0.2569,  1.3447, -1.0964, -0.7743,  0.4888],
         [ 0.2569,  1.3447, -1.0964, -0.7743,  0.4888]],

        [[-0.3436, -0.7069,  0.2946, -0.0270, -0.3178],
         [-0.3436, -0.7069,  0.2946, -0.0270, -0.3178]],

        [[ 0.2489, -0.2260, -0.0092, -0.4739,  0.2413],
         [ 0.2489, -0.2260, -0.0092, -0.4739,  0.2413]]],
       grad_fn=<StackBackward0>)
torch.Size([3, 2, 5]) 



In [87]:
'''
③torch.split()。
将tensor进行分隔，不改变形状。
第一个参数是tensor，
第二个参数是整数或者整数列表，代表需要划分的个数，
第三个参数是进行划分操作的维度，默认是0
'''

#将X从第1个维度按照2个元素为一组，返回含多个tensor的列表，如果最后一组不够2，则按照其实际数量形成一个tensor
print('X shape:',X.shape,'\n')
print('X splited0 shape:',torch.split(X,2,1)[0].shape)
#对划分的区间使用列表形式，[2,1]分别表示希望第0个tensor在维度1上有2个数据，以及第1个tensor在维度1上有1个数据
print('X splited1 shape:',torch.split(X,[2,1],1)[1].shape)

X shape: torch.Size([2, 3, 5]) 

X splited0 shape: torch.Size([2, 2, 5])
X splited1 shape: torch.Size([2, 1, 5])


#### 2、修改
#### tensor数据的修改  torch.where(), torch.index_fill(), torch.masked_fill()


①torch.where()。对满足条件和不满足条件的元素分别赋值。第一个参数为判断语句，将根据判断语句对数据集内每个元素进行运算。对于符合判断语句的参数，赋给第二个参数(tensor或者scalar)的值，对于不符合判断语句的参数，赋给第三个参数(tensor或者scalar)的值

②torch.index_fill()。对一个维度的目标索引进行直接赋值。对目标数据集(第一个参数)中的目标维度(第二个参数)中的目标索引tensor(第三个参数)，赋予给定的值(第四个参数，可以为标量或tensor)

③torch.masked_fill()。对所有符合条件的进行赋值。将目标数据集(第一个参数)中，满足判断语句(第二个参数)的所有元素，用一个给定的值(第三个参数)代替。

In [110]:
'''
①torch.where()。
第一个参数为判断语句，将根据判断语句对数据集内每个元素进行运算。
对于符合判断语句的参数，赋给第二个参数(tensor或者scalar)的值，
对于不符合判断语句的参数，赋给第三个参数(tensor或者scalar)的值
'''
#第二个和第三个参数可以是标量
print('id X:',id(X))
Y = torch.where(X>0,1,2)
print(Y)
print('id Y:',id(Y),'\n')

#第二个和第三个参数也可以是tensor
print(torch.where(X>0,torch.tensor(1),torch.tensor(2)))

id X: 2875858602080
tensor([[[2, 1, 1, 1, 1],
         [2, 2, 2, 1, 1],
         [1, 2, 1, 1, 2]],

        [[1, 1, 2, 2, 2],
         [1, 2, 1, 1, 2],
         [2, 1, 1, 1, 2]]])
id Y: 2875877576928 

tensor([[[2, 1, 1, 1, 1],
         [2, 2, 2, 1, 1],
         [1, 2, 1, 1, 2]],

        [[1, 1, 2, 2, 2],
         [1, 2, 1, 1, 2],
         [2, 1, 1, 1, 2]]])


In [118]:
'''
②torch.index_fill()。
对目标数据集(第一个参数)中的目标维度(第二个参数)中的目标索引tensor(第三个参数)，
赋予给定的值(第四个参数,可以为标量或tensor)
'''

#对目标数据集的第3个维度索引为0、1、2的数据，赋值0
print(torch.index_fill(X,2,torch.tensor([0,1,2]),0))

tensor([[[ 0.0000,  0.0000,  0.0000,  0.3553,  2.0771],
         [ 0.0000,  0.0000,  0.0000,  0.2399,  0.6537],
         [ 0.0000,  0.0000,  0.0000,  0.3208, -1.5857]],

        [[ 0.0000,  0.0000,  0.0000, -1.2062, -2.2592],
         [ 0.0000,  0.0000,  0.0000,  1.5264, -0.7635],
         [ 0.0000,  0.0000,  0.0000,  0.9923, -0.0999]]],
       grad_fn=<IndexFillBackward0>)


In [120]:
'''
③torch.masked_fill()。
对所有符合条件的进行赋值。
将目标数据集(第一个参数)中，满足判断语句(第二个参数)的所有元素，
用一个给定的值(第三个参数)代替。
'''
#将X中所有小于0的值变成0
print(X.masked_fill(X<0,0))

tensor([[[0.0000, 1.5211, 0.5001, 0.3553, 2.0771],
         [0.0000, 0.0000, 0.0000, 0.2399, 0.6537],
         [0.1136, 0.0000, 0.5320, 0.3208, 0.0000]],

        [[0.2852, 1.4625, 0.0000, 0.0000, 0.0000],
         [0.7998, 0.0000, 1.0541, 1.5264, 0.0000],
         [0.0000, 0.0989, 0.1931, 0.9923, 0.0000]]],
       grad_fn=<MaskedFillBackward0>)


#### 3、查询
#### 规则查询和提取 [ ], [,], [:::,], [...,]
和数组的索引操作基本一致，准确来说遵循如下原则

①如有多个方括号，第一个方括号表示对第一个维度的操作，第二个方括号表示对第二个维度的操作，依次类推

②方括号内如果有逗号，表示对多行进行操作，如果每个逗号范围内只有整数，则表示选择对应维度的索引。

③方括号内如果有逗号，且逗号范围内有':'，表示该维度存在切片操作，切片操作完整形式为(开始编号):(结束编号+1):(间隔)，且三个圆括号内都为int，都可选。

④省略号可以表示多个冒号

In [78]:
X = torch.randn((2,3,5),dtype=torch.float32,requires_grad= True)
print(X)
print(X.shape)

tensor([[[-0.7035, -1.3238, -0.2862,  0.3141, -0.4803],
         [-0.5608, -0.8724,  1.1259,  0.5028, -1.4117],
         [-0.7568, -0.2669,  0.7090,  0.3945,  0.3372]],

        [[-0.1660,  0.5910,  0.8902, -0.0372,  0.2757],
         [-0.4745,  0.1998, -0.9100,  0.5116, -1.3115],
         [-0.0752, -0.5022,  1.0219,  0.5539,  0.0771]]], requires_grad=True)
torch.Size([2, 3, 5])


In [85]:
'''
①如有多个方括号，第一个方括号表示对第一个维度的操作，
第二个方括号表示对第二个维度的操作，依次类推
'''

#选取第一个维度
X[0]

tensor([[-0.7035, -1.3238, -0.2862,  0.3141, -0.4803],
        [-0.5608, -0.8724,  1.1259,  0.5028, -1.4117],
        [-0.7568, -0.2669,  0.7090,  0.3945,  0.3372]],
       grad_fn=<SelectBackward0>)

In [83]:
#选取第二个维度
print(X[0][0])

#或者是:
'''
②方括号内如果有逗号，表示对多行进行操作，
如果每个逗号范围内只有整数，则表示选择对应维度的索引。
'''
print(X[0,0])

tensor([-0.7035, -1.3238, -0.2862,  0.3141, -0.4803],
       grad_fn=<SelectBackward0>)
tensor([-0.7035, -1.3238, -0.2862,  0.3141, -0.4803],
       grad_fn=<SelectBackward0>)


In [84]:
#选取第三个维度
print(X[0][0][0])
#或者是
print(X[0,0,0])

tensor(-0.7035, grad_fn=<SelectBackward0>)
tensor(-0.7035, grad_fn=<SelectBackward0>)


In [86]:
'''
③方括号内如果有逗号，且逗号范围内有':'，
表示该维度存在切片操作，
切片操作完整形式为(开始编号):(结束编号+1):(间隔)，
三个圆括号内都为int，且都可选。
'''
#获取第一个维度从索引1开始的所有，第二个维度的所有，第三个维度从0到3（不包含4）间隔为2的所有数据
X[1::,::,0:4:2]

tensor([[[-0.1660,  0.8902],
         [-0.4745, -0.9100],
         [-0.0752,  1.0219]]], grad_fn=<SliceBackward0>)

In [87]:
'''
④省略号可以代替多个冒号
'''
X[1::,...,0:4:2]

tensor([[[-0.1660,  0.8902],
         [-0.4745, -0.9100],
         [-0.0752,  1.0219]]], grad_fn=<SliceBackward0>)

#### 不规则查询和提取  torch.index_select(), torch.take(), torch.masked_select()


①torch.index_select()。自动保持维度不变。第一个参数为待选取的tensor数据集，第二个参数为需要选取的维度编号，第三个参数为第二个参数代表维度下需要选取的索引，如果需要对多个维度进行选取，可以将该方法嵌套使用

②torch.take()。维度会发生变化。第一个参数为待选取的tensor数据集，第二个参数为待选取的数据在数据集中的序号组成的tensor对象，可以包含多个序号，这里的序号指的是将目标数据集看做一维之后，目标数据在一维中的编号。

③torch.masked_select()。维度会发生变化。第一个参数为待选取的tensor数据集，第二个参数为一个数据集的条件判断语句，如：tensor>0，其中tensor为待选取的数据集

In [89]:
X = torch.randn((2,3,5),dtype=torch.float32,requires_grad= True)
print(X)
print(X.shape)

tensor([[[-0.0651,  1.5211,  0.5001,  0.3553,  2.0771],
         [-0.3993, -0.1759, -0.3652,  0.2399,  0.6537],
         [ 0.1136, -0.6993,  0.5320,  0.3208, -1.5857]],

        [[ 0.2852,  1.4625, -1.5551, -1.2062, -2.2592],
         [ 0.7998, -0.5394,  1.0541,  1.5264, -0.7635],
         [-0.6598,  0.0989,  0.1931,  0.9923, -0.0999]]], requires_grad=True)
torch.Size([2, 3, 5])


In [98]:
'''
①torch.index_select()。
第一个参数为待选取的tensor数据集，
第二个参数为需要选取的维度编号，
第三个参数为第二个参数代表维度下需要选取的索引，
如果需要对多个维度进行选取，可以将该方法嵌套使用
'''
#选择第1个维度的第1个数据
print(torch.index_select(X, 1, torch.tensor(1)))
print(X.index_select(1, torch.tensor(1)).shape,'\n\n')

#对第1个维度的第1个数据和第2个维度的第2个数据进行选取
#因为维度保持不变，所以嵌套使用时，选取维度时还是要参考X数据的维度
print(torch.index_select(torch.index_select(X, 1, torch.tensor(1)),2,torch.tensor([0,2,4])))
print(torch.index_select(torch.index_select(X, 1, torch.tensor(1)),2,torch.tensor([0,2,4])).shape)

tensor([[[-0.3993, -0.1759, -0.3652,  0.2399,  0.6537]],

        [[ 0.7998, -0.5394,  1.0541,  1.5264, -0.7635]]],
       grad_fn=<IndexSelectBackward0>)
torch.Size([2, 1, 5]) 


tensor([[[-0.3993, -0.3652,  0.6537]],

        [[ 0.7998,  1.0541, -0.7635]]], grad_fn=<IndexSelectBackward0>)
torch.Size([2, 1, 3])


In [102]:
'''
②torch.take()。
第一个参数为待选取的tensor数据集，
第二个参数为待选取的数据在数据集中的序号组成的tensor对象，
可以包含多个序号，
这里的序号指的是将目标数据集看做一维之后，目标数据在一维中的编号。
'''

#抽取X数据集中的第0、3、5个数据（按一维计算）
print(X.take(torch.tensor([0,3,5])))
print(torch.take(X,torch.tensor([0,3,5])))

tensor([-0.0651,  0.3553, -0.3993], grad_fn=<TakeBackward0>)
tensor([-0.0651,  0.3553, -0.3993], grad_fn=<TakeBackward0>)


In [104]:
'''
③torch.masked_select()。
第一个参数为待选取的tensor数据集，
第二个参数为一个数据集的条件判断语句，
如：tensor>0，其中tensor为待选取的数据集
'''

#选取X中所有大于0的元素
print(X.masked_select(X>0))

#选取X中所有小于0的元素
print(torch.masked_select(X,X<0))

tensor([1.5211, 0.5001, 0.3553, 2.0771, 0.2399, 0.6537, 0.1136, 0.5320, 0.3208,
        0.2852, 1.4625, 0.7998, 1.0541, 1.5264, 0.0989, 0.1931, 0.9923],
       grad_fn=<MaskedSelectBackward0>)
tensor([-0.0651, -0.3993, -0.1759, -0.3652, -0.6993, -1.5857, -1.5551, -1.2062,
        -2.2592, -0.5394, -0.7635, -0.6598, -0.0999],
       grad_fn=<MaskedSelectBackward0>)


#### 4、形状调整
#### tensor数据形状调整  torch.reshape()&torch.squeeze()/torch.unsqueeze(), torch.transpose()

①torch.reshape()&torch.squeeze()/torch.unsqueeze()。reshape根据传入的维度形状对数据集进行变形,传递的维度形状中可以有一个数值为-1，为-1的维度将由库自动计算得出。增加元素数量为0的维度需要用到unsqueeze，减少元素数量为0的维度需要用到squeeze

②torch.transpose()。和numpy的transpose能依次指定每个轴的顺序不太一样，此处的transpose只能指定前两个轴进行交换，且不对原对象进行修改，会返回一个新的对象

In [90]:
'''
①torch.reshape()。根据传入的维度形状对数据集进行变形

'''
Y = torch.reshape(X,(30,-1))
print('X.shape:',X.shape)
print('Y.shape:',Y.shape)

X.shape: torch.Size([2, 3, 5])
Y.shape: torch.Size([30, 1])


In [71]:
'''
②torch.transpose()。只能指定对哪两个维度进行交换。
且不影响原变量，而是返回一个新的对象

'''
#对第2个维度和第0个维度进行交换
Y = torch.transpose(X,2,0)
print('X.shape:',X.shape)
print('Y.shape:',Y.shape)

X.shape: torch.Size([2, 3, 5])
Y.shape: torch.Size([5, 3, 2])


In [42]:
#numpy中的transpose可以指定所有轴的次序
Z = X.detach().numpy()
print('Z.shape:',Z.shape)
W = Z.transpose(2,0,1)
print('W.shape:',W.shape)

Z.shape: (2, 3, 5)
W.shape: (5, 2, 3)


### 四、tensor数据运算


#### 1、标量运算

加减乘除余数等

torch.max()   
求每个位置的最大值

torch.min()   
求每个位置的最小值

torch.round()   
求整数:每个位置四舍五入

torch.floor()   
求整数：每个位置靠近的两个整数中小的那一个

torch.ceil()   
求整数：每个位置靠近的两个整数中大的那一个

torch.trunc()   
求整数：每个位置直接把小数部分去掉

torch.fmod()   
求余数：余数有正有负

torch.remainder()   
求余数：余数一定为正

torch.clamp()   
限定最大和最小值，超出的改成最大或最小值

In [145]:
a = torch.randn(2,2)
b = torch.randn(2,2)
print('a:',a)
print('b:',b,'\n')
print('torch.max(a,b):\n',torch.max(a,b),'\n') #返回两个或多个数据集在每个位置上的最大值
print('torch.min(a,b):\n',torch.min(a,b),'\n') #返回两个或多个数据集在每个位置上的最小值
print('torch.round(a):\n',torch.round(a),'\n') #返回一个数据集所有元素四舍五入之后的整数值
print('torch.floor(a):\n',torch.floor(a),'\n') #返回一个数据集中所有元素的靠近每个元素的两个整数中较小的那个整数
print('torch.ceil(a):\n',torch.ceil(a),'\n') #返回一个数据集中所有元素的靠近每个元素的两个整数中较大的那个整数
print('torch.trunc(a):\n',torch.trunc(a),'\n') #返回一个数据集中所有元素的整数部分
print('torch.fmod(a,1):\n',torch.fmod(a,1),'\n') #除法取余数部分，余数可以为正数也可以为负数
print('torch.remainder(a,1):\n',torch.remainder(a,1),'\n') #除法取余数部分，所有的余数都要求是正数，如果元素是负数，需要先加n倍分母至最小的正数，然后取余数
print('torch.clamp(a,min=-1,max=1):',torch.clamp(a,min=-1,max=1)) #将数据集中超过指定最大值或最小值范围的元素改成给定的最大或最小值


a: tensor([[0.2635, 0.6008],
        [0.0630, 1.0336]])
b: tensor([[ 0.0929,  1.0667],
        [ 2.2848, -0.2672]]) 

torch.max(a,b):
 tensor([[0.2635, 1.0667],
        [2.2848, 1.0336]]) 

torch.min(a,b):
 tensor([[ 0.0929,  0.6008],
        [ 0.0630, -0.2672]]) 

torch.round(a):
 tensor([[0., 1.],
        [0., 1.]]) 

torch.floor(a):
 tensor([[0., 0.],
        [0., 1.]]) 

torch.ceil(a):
 tensor([[1., 1.],
        [1., 2.]]) 

torch.trunc(a):
 tensor([[0., 0.],
        [0., 1.]]) 

torch.fmod(a,1):
 tensor([[0.2635, 0.6008],
        [0.0630, 0.0336]]) 

torch.remainder(a,1):
 tensor([[0.2635, 0.6008],
        [0.0630, 0.0336]]) 

torch.clamp(a,min=-1,max=1): tensor([[0.2635, 0.6008],
        [0.0630, 1.0000]])


#### 2、多维向量运算

torch.max()   
求一个或多个维度上的最大值

torch.min()   
求一个或多个维度上的最小值

torch.sum()   
求一个维度或多个维度上的和

torch.mean()   
求一个维度或多个维度上的平均值

torch.prod()   
求一个或多个维度上的累加值

torch.std()   
求一个或多个维度上的标准差

torch.var()   
求一个或多个维度上的方差

torch.median()  
求一个或多个维度上的中位数

torch.cumsum() 

torch.cumprod() 

torch.cummax()

torch.cummin()

torch.topk()  
返回指定维度下最大的k个值，第一个参数是tensor,第二个参数是k的值，第三个参数是维度的值

torch.sort()  
对指定维度进行排序，可以通过关键字说明是否为倒序

@/torch.matmul()/torch.mm() 
矩阵乘积，注意矩阵只能是二维的

tensor.t()/tensor.T  
矩阵转置

#### 3、爱因斯坦运算

顾名思义，einsum这个函数的思想起源于家喻户晓的小爱同学：爱因斯坦~。

很久很久以前，小爱同学在捣鼓广义相对论。广义相对论表述各种物理量用的都是张量。

比如描述时空有一个四维时空度规张量，描述电磁场有一个电磁张量，描述运动的有能量动量张量。

在理论物理学家中，小爱同学的数学基础不算特别好，在捣鼓这些张量的时候，他遇到了一个比较头疼的问题：公式太长太复杂了。

有没有什么办法让这些张量运算公式稍微显得对人类友好一些呢，能不能减少一些那种扭曲的$\sum$求和符号呢？

小爱发现，求和导致维度收缩，因此求和符号操作的指标总是只出现在公式的一边。

例如在我们熟悉的矩阵乘法中

$$C_{ij} = \sum_{k} A_{ik} B_{kj}$$

k这个下标被求和了，求和导致了这个维度的消失，所以它只出现在右边而不出现在左边。

这种只出现在张量公式的一边的下标被称之为哑指标，反之为自由指标。

小爱同学脑瓜子一转，反正这种只出现在一边的哑指标一定是被求和求掉的，干脆把对应的$\sum$求和符号省略得了。

这就是爱因斯坦求和约定：

 <font color="red">**只出现在公式一边的指标叫做哑指标，针对哑指标的$\sum$求和符号可以省略。** </font> 

公式立刻清爽了很多。

$$C_{ij} =  A_{ik} B_{kj}$$

这个公式表达的含义如下：

C这个张量的第i行第j列由$A$这个张量的第i行第k列和$B$这个张量的第k行第j列相乘，这样得到的是一个三维张量$D$, 其元素为$D_{ikj}$，然后对$D$在维度k上求和得到。

公式展现形式中除了省去了求和符号，还省去了乘法符号(代数通识)。


借鉴爱因斯坦求和约定表达张量运算的清爽整洁，numpy、tensorflow和 torch等库中都引入了 einsum这个函数。

上述矩阵乘法可以被einsum这个函数表述成


```python
C = torch.einsum("ik,kj->ij",A,B)
```

这个函数的规则原理非常简洁，3句话说明白。

<font color="red">
   
* 1，用元素计算公式来表达张量运算。

* 2，只出现在元素计算公式箭头左边的指标叫做哑指标。

* 3，省略元素计算公式中对哑指标的求和符号。
    
</font> 



In [147]:
A = torch.tensor([[1,2],[3,4.0]])
B = torch.tensor([[5,6],[7,8.0]])

C1 = A@B
print(C1)

C2 = torch.einsum("ik,kj->ij",[A,B])
print(C2)

tensor([[19., 22.],
        [43., 50.]])
tensor([[19., 22.],
        [43., 50.]])


In [148]:
#例1，张量转置
A = torch.randn(3,4,5)

#B = torch.permute(A,[0,2,1])
B = torch.einsum("ijk->ikj",A) 

print("before:",A.shape)
print("after:",B.shape)

before: torch.Size([3, 4, 5])
after: torch.Size([3, 5, 4])


In [149]:
#例3，求和降维
A = torch.randn(4,5)
#B = torch.sum(A,1)
B = torch.einsum("ij->i",A)
print("before:",A.shape)
print("after:",B.shape)

before: torch.Size([4, 5])
after: torch.Size([4])


In [150]:
#例4，哈达玛积
A = torch.randn(5,5)
B = torch.randn(5,5)
#C=A*B
C = torch.einsum("ij,ij->ij",A,B)
print("before:",A.shape, B.shape)
print("after:",C.shape)

before: torch.Size([5, 5]) torch.Size([5, 5])
after: torch.Size([5, 5])


In [ ]:
torch.nn.Parameter(tensor)
torch.nn.ParameterDict(key1:torch.nn.Parameter(tensor),key2:torch.nn.Parameter(tensor))
torch.nn.ParameterList(torch.nn.Parameter(tensor),torch.nn.Parameter(tensor))

torch.nn.Module()
torch.nn.ModuleDict(torch.nn.Module/网络层/sequrntial,……) #返回一个模型，传入的参数是各种模型层或网络层
torch.nn.ModuleList(torch.nn.Module/网络层/sequrntial,……)

torch.nn.Module().attr1 = torch.nn.Embedding()  #创建一个网络层,将其赋值给一个.Module类对象的属性

torch.nn.Module().attr2 =torch.nn.Sequential()  #创建一个放置网络层的容器，与网络层平级，将其作为一个模型对象的属性值

torch.nn.Sequential.add_module()  #向容器中添加模型
torch.nn.Conv1d()  #各种不同的神经网络模型，设置参数个数，然后实例化同时也可以是一个单独的网络层
torch.nn.Conv2d()
torch.nn.Conv3d()
torch.nn.MaxPool1d()  #…………
torch.nn.ReLU()
torch.nn.Flatten()
torch.nn.Linear()  #…………
torch.nn.Sequential()

torch.nn.Module.children()  #返回模型中所有容器包含的层的信息
torch.nn.Module.named_children()  #返回模型中所有容器的名字和其包含的层的信息

torch.nn.Module.parameters() #返回模型所有的attrs，即torch.nn.Module().attr1……

summary() #torchkears.summary()显示模型详细信息


#所有的网络层都是以模型的属性而存在，有下面三类形式

#1、每个网络层作为模型的一个属性
class Net(torch.nn.Module):
    def __init__(self,.....):
        super().__init__()
        self.layer1 = torch.nn.Module.Conv1d(....)
        self.layer2 = torch.nn.Module.Flatten(...)
        ......
    def forward(self,x):  #x就是输入数据
        x = self.layer1(x)
        y = self.layer2(x)
        return y  #返回最终的loss数据
    
    
#2、将网络层放在一个容器中，容器内的层会自动的进行forward，适用于简单的网络定义
#①直接实例化一个容器，然后添加网络层
net = torch.nn.Sequential()
net.add_module('str of model name',torch.nn.Module.Conv1d())
net.add_module('str of model name',torch.nn.Module.Conv2d())
net.add_module('str of model name',torch.nn.Module.Conv3d())
#②用长参数直接指定网络层内容和顺序，但是不能给网络层命名
net = torch.nn.Sequential(
    torch.nn.Module.Conv1d()
    torch.nn.Module.Conv2d()
    torch.nn.Module.Conv3d()
) 
#③用OrderedDict来组成有序的字典模型，解决方法2不能命名的问题
net = torch.nn.Sequential(OrderedDict(
    [('str of model name',torch.nn.Module.Conv1d())
     ('str of model name',torch.nn.Module.Conv2d())
     ('str of model name',torch.nn.Module.Conv3d())
    ])
)


#3、将多个网络层防在多个容器里，将每个容器作为模型的一个属性
#①用sequential作为容器，因为它可以自动对容器内的网络层进行前向传播，所以最方便
class Net(torch.nn.Module):
    def __init__(self,.....):
        super().__init__()
        self.conv  = torch.nn.Sequential()
        self.conv.add_module('str of model name',torch.nn.Module.Conv1d())
        self.conv.add_module('str of model name',torch.nn.Module.Conv2d())
        self.conv.add_module('str of model name',torch.nn.Module.Conv3d())
        sellf.dense = torch.nn.Sequential()
        self.dense.add_module('str of model name',torch.nn.Module.Conv1d())
        self.dense.add_module('str of model name',torch.nn.Module.Conv2d())
        self.dense.add_module('str of model name',torch.nn.Module.Conv3d())
    def forward(self,x):
        x = self.conv(x)
        y = self.dense(x)
        return y
#②用ModuleList作为容器，需要写循环对每个网络层进行前向传播
class Net(torch.nn.Module):
    def __init__(self,....):
        super().__init__()
        self.layers = torch.nn.ModuleList(
        [torch.nn.Module.Conv1d(),
         torch.nn.Module.Conv2d(),
         torch.nn.Module.Conv3d()
        ])
    def forward(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
#③使用ModuleDict作为容器。
#  需要将所有的key按需要的顺序组成一个列表，然后对所有key进行循环以前向传播
class Net(torch.nn.Module):
    def __init__(self,...):
        super().__init__()
        slef.layers =torch.nn.ModuleDict(
        {'modelName1':torch.nn.Module.Conv1d(),
         'modelName2':torch.nn.Module.Conv2d(),
         'modelName3':torch.nn.Module.Conv3d()
        })
    def forward(self,x):
        li= ['modelName1','modelName2','modelName3']
        for modelName in li:
            x=self.layers[modelName](x)
        return x

#
KerasModel.fit() 



In [59]:
'''
CUDA的使用
需要数据和模型都在cuda上
'''

#①数据如何放在cuda上
import torch

print(torch.cuda.is_available()) #判断机器上是否有cuda
print(torch.cuda.device_count()) #判断有几个cuda

cpu_device = torch.device('cpu')     #创建一个cpu设备
cuda_device = torch.device('cuda:0') #创建一个cuda设备

cpu_tensor = torch.randn(20,10)                       #创建一个cpu上的tensor
cuda_tensor = torch.randn((20,10),device=cuda_device) #创建一个cuda上的tensor

print(cpu_tensor.device)   #查看创建的cpu上的tensor是否在cpu上
print(cuda_tensor.device)  #查看创建的cuda上的tensor是否在cuda上

cuda_tensor2 = cpu_tensor.to(cuda_device) #将cpu上的tensor转移到cuda上，两种方式都可以
cuda_tensor3 = cpu_tensor.cuda()          #将cpu上的tensor转移到cuda上，两种方式都可以

print(cuda_tensor2.device)  #查看创建的cuda上的tensor2是否在cuda上
print(cuda_tensor3.device)  #查看创建的cuda上的tensor3是否在cuda上

cpu_tensor2 = cuda_tensor.to(cpu_device) #将cuda上的tensor转移到cpu上，两种方式都可以
cpu_tensor3 = cuda_tensor.cpu()          #将cuda上的tensor转移到cpu上，两种方式都可以

print(cpu_tensor2.device)  #查看创建的cpu上的tensor2是否在cpu上
print(cpu_tensor3.device)  #查看创建的cpu上的tensor3是否在cpu上

#②模型如何放在cuda上
net = torch.nn.Module()                        #创建一个模型
net.w = torch.nn.Parameter(torch.zeros(3,10))  #通过观察模型的参数是否在cuda上的形式来判断模型是否转移成功，所以这里添加了一个参数，方便后面进行验证
net = net.to(cuda_device)                      #将模型移动到cuda上
print('if on cuda:',next(net.parameters()).is_cuda)  #验证模型的参数是否在cuda上

True
1
cpu
cuda:0
cuda:0
cuda:0
cpu
cpu
if on cuda: True


#### 从大数据集中提取一个小批次的随机数据，以供下一步模型进行训练
又称为random_mini_batch_size。这个批次数据的数量称为batch_size,批次数据数量并不大，所以称为mini_batch_size。另外，这个批次的数据已经是打散的了，打散的目的是为了解决原始数据集中，相邻数据样本往往都是同类型的样本的情况，这种情况往往会导致模型在一段时间内因为接受同样类型的数据，不能进行优化的问题

In [ ]:
import numpy as np
import torch

#创建一个数据分批次生成器
#参数有三个，分别是数据集，标签，以及批次数据的大小
def data_iter(features, labels, batch_size= 'defalut'):
    
    #计算数据集的长度
    num_examples = len(features) 
    
    #生成一个包含所有数据索引号的列表
    indices = list(range(num_examples)) 
    
    #利用numpy的.random.shuffle功能将所有的索引号打乱，注意，此处生成的是一个numpy对象
    np.random.shuffle(indices) 
    
    #创建batch_size规模的生成器
    for i in range(0, num_examples, batch_size): 
        
        #首先取一个batch_size的所有序号，注意取的时候要跟数组的最大序号比较，不能超出最大序号，所以用了min()
        #然后将所有的numpy数据转换成torch.int64类型，.LongTensor()的作用是将numpy数据转换成tensor数据
        #需要使用.LongTensor()的原因是，后面的torch.index_select()函数其中一个参数要求必须是LongTensor类型的
        indexes = torch.LongTensor(indices[i : min(i + batch_size, num_examples)])
        
        #使用torch.index_select()函数，根据数据的索引生成数据
        #torch.index_select(参数1 params：待选取的数据,
        #                     参数2 int:在待选取数据的哪个维度上进行选取,
        #                     参数3 torch.LongTensor:需要被选中的数据的索引，参数类型要求为tensor的整数型)
        #                     参数4 out=tensorName :可选参数，可以将生成的结果通过明确‘out=’的形式返回写入到一个tensor
        yield torch.index_select(features, 0, indexes), torch.index_select(labels, 0, indexes)

#### 利用torch.utils.data下的TensorDataset和DataLoader生成batch

TensorDataset：   
可以传入多个tensor(如训练数据和label可以作为两个tensor一起传入)，返回一个对象，这个对象将按照传入数据的第0个维度为目标，根据索引输出数据。

DataLoader：   
能够控制batch的大小，batch中元素的采样方法，以及将batch结果整理成模型所需输入形式的方法，并且能够使用多进程读取数据。

```python
DataLoader(
    dataset,  #数据集
    batch_size=1,   #批次大小
    shuffle=False,  #是否乱序
    sampler=None,   #样本采样函数，一般无需设置。
    batch_sampler=None,   #批次采样函数，一般无需设置。
    num_workers=0,  #使用多进程读取数据，设置的进程数。
    collate_fn=None, #整理一个批次数据的函数。
    pin_memory=False,#是否设置为锁业内存。默认为False，锁业内存不会使用虚拟内存(硬盘)，从锁业内存拷贝到GPU上速度会更快。
    drop_last=False, #是否丢弃最后一个样本数量不足batch_size批次数据。
    timeout=0,     #加载一个数据批次的最长等待时间，一般无需设置。
    worker_init_fn=None, #每个worker中dataset的初始化函数，常用于 IterableDataset。一般不使用。
    multiprocessing_context=None,
)
```


In [36]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

a=torch.randn(10,4)
b=torch.randn(10).round()
print(a)
print(b,'\n')

ds = TensorDataset(a,b)
dl=DataLoader(ds,batch_size=3,shuffle=True,drop_last=False)
#print(list(dl)) DataLoader返回的是一个类list对象，需要通过for循环进行读取

for i,(batch,label) in enumerate(dl):
    print(f'batch{i}')
    print('train:',batch,)
    print('label:',label,'\n')

tensor([[-0.3588,  0.5223,  0.3082,  1.4179],
        [-0.5506,  2.1220,  0.4137, -0.1276],
        [ 0.0302,  0.0172,  2.2157,  1.5716],
        [-0.3546,  0.8855,  0.2870, -1.0324],
        [ 0.0562, -0.4361, -0.5903, -2.6593],
        [ 0.2717, -0.0872, -1.3222,  1.0993],
        [-0.4902,  0.1126,  0.2516, -1.2350],
        [ 0.1216, -0.0113,  0.3717, -0.1140],
        [-0.1222, -0.2311, -1.7783,  0.1156],
        [ 0.3714,  0.4311, -0.1355, -0.8100]])
tensor([-1.,  2.,  0., -0., -1.,  2.,  0.,  0.,  0.,  1.]) 

batch0
train: tensor([[ 0.1216, -0.0113,  0.3717, -0.1140],
        [ 0.0302,  0.0172,  2.2157,  1.5716],
        [-0.4902,  0.1126,  0.2516, -1.2350]])
label: tensor([0., 0., 0.]) 

batch1
train: tensor([[-0.3588,  0.5223,  0.3082,  1.4179],
        [-0.3546,  0.8855,  0.2870, -1.0324],
        [ 0.0562, -0.4361, -0.5903, -2.6593]])
label: tensor([-1., -0., -1.]) 

batch2
train: tensor([[-0.5506,  2.1220,  0.4137, -0.1276],
        [ 0.3714,  0.4311, -0.1355, -0.8100],
   